# Chapter 05 트리 알고리즘

### 5-1 결정 트리

알코올, 도수, 당도, pH값으로 와인 분류하기. ( Red or White )

로지스틱 회귀로 분류하기:

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

# class: 0 --> 레드 와인
# class: 1 --> 화이트 와인

In [ ]:
wine.info()  # info(): 데이터프레임 각 열의 데이터 타입과 누락된 데이터가 있는지 확인

In [ ]:
wine.describe()  # describe(): 열에 대한 간략한 통계 출력 (최소, 최대, 평균, 표준편차 등...)

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()  # 각 특성 데이터
target = wine['class'].to_numpy()   # 타겟값

In [ ]:
# 훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
print(train_input.shape, test_input.shape)

In [ ]:
# 특성 스케일링 (정규화)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 로지스틱 회귀 모델 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

In [ ]:
print(lr.coef_, lr.intercept_)

각 coef에 특성을 곱해서 더하고 intercept를 더하면 z값이 됨

그걸 시그모이드함수에 넣고 0보다 크면 양성(화이트와인), 작으면 음성(레드와인) 이다.

결정 트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
# 결정트리 모델 그리기
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

In [ ]:
# 위 그림이 복잡하니 확대해서 그리기
plt.figure(figsize=(10,7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
# filled=True: 색 채우기 --> 어떤 클래스의 비율이 높아지면 점점 진한 색으로 표시
plt.show()

왼쪽이 Yes, 오른쪽이 No

- 테스트 조건 (sugar)

- 불순도 (gini)

- 총 샘플 수 (samples)

- 클래스별 샘플 수 (value) : (음성클래스 수, 양성클래스 수)

결정트리에서 예측하는 방법 : 리프노드에서 가장 많은 클래스가 예측 클래스가 된다

---

**gini ( 지니 불순도 )** : 테스트 조건을 결정

지니 불순도 = 1 - ( 음성클래스 비율²+ 양성클래스 비율² )

- 불순도 == 0인 노드 : 순수 노드

결정트리 모델은 부모노드와 자식녿의 불순도 차이가 가능한 크도록 트리를 성장시킨다!

--> 불순도 차이:

부모의 불순도 - (왼쪽노드 샘플수 / 부모의 샘플수) x 왼쪽노드 불순도 - (오른쪽노드 샘플수 / 부모의 샘플수) x 오른쪽노드 샘플수

불순도 차이를 **정보이득** 이라고 한다.

즉, 결정트리는 정보이득이 최대가 되도록 데이터를 나눈다, 노드를 순수하게 나눌수록 정보이득이 최대가 된다.

새로운 샘플을 예측할 때에는 노드의 질문에 따라 트리를 이동하여 마지막에 도달한 노드의 클래스 비율을 보고 예측한다!

*엔트로피 불순도* 는 클래스비율과 클래스비율에 밑이 2인 로그를 적용한 값을 곱해서 모두 더한 후 음수로 바꾸어 계산 (default=gini)

In [ ]:
# 가지치기 : 결정트리가 끝까지 성장하는 것을 제한
dt = DecisionTreeClassifier(max_depth=3, random_state=42) # max_depth=3 으로 제한
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

3번째 노드만 음성클래스가 더 많다 --> 이 노드에 도착해야만 레드와인으로 예측한다.

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

# 특성값의 스케일은 결정트리 알고리즘에 아무런 영향을 미치지 않는다
# --> 표준화 전처리를 할 필요가 없다!..
# 결과 같음

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

In [ ]:
print(dt.feature_importances_)  # 특성 중요도 출력
# 루트노드와 depth=1 에서 sugar을 사용했기 때문에 sugar의 중요도가 제일 높을것으로 예상
# --> 결정트리의 특성 중요도를 특성 선택에 활용할 수 있다!

### 5.2 교차 검증과 그리드 서치

결정 트리에서 최적의 max_depth를 찾기 위해서는 여러 번 테스트를 해야한다

--> 테스트세트에 과대적합할 우려가 있다.. ( 테스트 세트는 한 번만 사용하는 것이 좋다 )

--> 훈련세트를 나누면 된다 : ( **검증 세트** )

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# 똑같이 훈련세트에서 검증세트를 뽑아낸다
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

교차 검증:

검증세트를 떼어 내어 평가하는 과정을 여러 번 반복하여 그 점수를 평균하여 최종 검증 점수를 얻는다.

--> 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다!

In [ ]:
from sklearn.model_selection import cross_validate      # 교차검증 함수
scores = cross_validate(dt, train_input, train_target)  # 훈련세트 전체를 전달
                                                        # 기본적으로 5-폴드 교차검증을 한다 (cv매개변수로 변경가능)
print(scores)

# 'fit_time', 'score_time' : 훈련하는 시간, 검증하는 시간
# 'test_score' : 검증 폴드의 점수

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))   # 검증 폴드 점수의 평균

In [ ]:
# cross_validate()는 훈련세트를 섞어 폴드를 나누지 않아서 (train_test_split()에서 이미 섞었기 때문)
#                   교차검증을 할 때 섞으려면 사이킷런 분할기를 사용해야 한다.
from sklearn.model_selection import StratifiedKFold   # 분류모델의 경우 StratifiedKFold, 회귀모델이면 'KFold' 분할기 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # n_splits=k : k-폴드 교차검증 시행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

# KFold 클래스도 사용

**하이퍼파라미터** 튜닝

- 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터

- 하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 ( 사용자 지정 파라미터 )

예를 들어, 두 하이퍼파라미터의 최적값을 찾아야한다면, 하나 찾고 나머지를 찾는 것이 아니라, 동시에 바꿔가면 최적의 값을 찾아야 한다!

- 사이킷런의 **GridSearchCV** 클래스 ( 그리드 서치 ) :

 하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다. ( 별도로 cross_validate( )를 호출할 필요X )

In [ ]:
from sklearn.model_selection import GridSearchCV

# 결정트리 모델에서 min_impurity_decreas 매개변수의 최적값 찾기
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)  # n_jobs : 병렬실행에 사용할 CPU 코어 수
#                                                                                 기본값=1,  -1이면 시스템에 있는 모든 코어 사용

In [ ]:
gs.fit(train_input, train_target)  # 그리드 서치 훈련

# 그리드 서치 훈련이 끝나면 최적의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련해준다!

In [ ]:
dt = gs.best_estimator_   # best_estimator_ : 검증점수가 가장 높은 모델이 저장되어 있다 (최적의 모델)
print(dt.score(train_input, train_target))

In [ ]:
print(gs.best_params_)   # best_params_: 최적의 매개변수가 저장되어 있다

In [ ]:
print(gs.cv_results_['mean_test_score'])  # 5번의 교차검증으로 얻은 점수

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])   

# np.argmax() : 가장 큰 값의 인덱스 추출

그리드 서치 과정:

1. 먼저 탐색할 매개변수를 지정

2. 훈련세트에서 그리드 서치 수행하여 초상의 평균 검증점수가 나오는 매개변수 조합을 찾는다 ( 이 조합은 그리드서치 객체에 저장됨 )

3. 그리드서치는 자동으로 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련한다 ( 이 모델도 그리드서치 객체에 저장됨 )

In [ ]:
# 조금 더 복잡한 매개변수 조합

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # np.arange() : 실수도 가능
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

# 'min_impurity_decrease': 노드를 분할하기 위한 불순도 감소 최소량 지정
# 'max_depth': 트리의 깊이 제한
# 'min_samples_split': 노들르 나누기 위한 최소 샘플 수

# 총 교차 검증 횟수: 9 * 15 * 10 = 1350번
#         --> 기본 5-폴드 교차검증이므로 6750개의 모델이 만들어진다!
len(gs.cv_results_['mean_test_score'])

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)  # 최상의 매개변수 조합

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

랜덤 서치 :

매개변수의 값의 범위나 간격을 정하기 어려울 때 사용

매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

싸이파이 : 파이썬 수치 계산 전용 라이브러리

In [ ]:
# 싸이파이의 확률 분포 클래스
from scipy.stats import uniform, randint   # 주어진 범위에서 고르게 값을 뽑는 클래스
#                                            '균등 분포에서 샘플링한다'
#                                          uniform: 실수값, randint: 정수값

rgen = randint(0, 10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)  # 1000개를 샘플링해서 각 숫자의 개수를 셈

# 어느 정도 고르게 분포되어 있다

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)   # 실수값 반환

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20,50),
         'min_samples_split': randint(2,25),
         'min_samples_leaf': randint(1,25)}

# 'min_samples_leaf': 리프노드가 되기 위한 최소 샘플의 개수

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)     # n_iter 만큼 샘플링한다
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))    # 최고의 교차 검증 점수

In [ ]:
dt = gs.best_estimator_     # 최적의 모델
print(dt.score(test_input, test_target))

### 5.3 트리의 앙상블

- 정형 데이터 : 어떤 구조로 되어 있는 데이터 ( CSV , 뎅터베이스 , 엑셀 등에 저장하기 쉽다 )

- 비정형 데이터 : 데이터베이스나 엑셀로 표현하기 어려운 것들

정형 데이터에 뛰어난 성과를 보이는 알고리즘 : **앙상블 학습**

비정형 데이터에 뛰어난 성과를 보이는 알고리즘 : **신경망 알고리즘**

---

**랜덤 포레스트** : 앙상블 학습의 대표 주자

결정 트리를 랜덤하게 만들어 결정트리의 숲을 만든다.

우선 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, '부트스트랩' 방식으로 샘플을 뽑는다. ( 부트스트랩 샘플 )

- 부트스트랩 : 데이터세트에서 중복을 허용하여 데이터를 샘플링하는 방식 , 즉, 뽑은 데이터를 다시 뽑을 수도 있다.

각 노드를 분할할 때에 전체 특성 중 일부 특성을 무작위로 고른 후, 이 중에서 최선의 분할을 찾는다.

분류모델 RandomForestClassifier 는 전체 특성 개수의 제곱근만큼의 특성을 선택한다. ( 회귀모델 RandomForestRegressor 는 전체 다 사용 )

In [ ]:
# 데이터 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)  
    # return_train_score=True: 검증세트 점수 뿐만 아니라 훈련세트에 대한 점수도 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)  # 결정트리의 특성 중요도를 취합

# 각 특성은 순서대로 'alcohol', 'sugar', 'pH' 이다

RandomForestClassifier 에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다

--> 부트스트랩 샘플에 포함되지 않은 샘플 (OOB 샘플) 을 가지고 훈련한 결정트리를 평가 (검증세트의 역할)

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)